In [0]:
import numpy as np
import gzip
import pickle as pkl
import matplotlib.pyplot as plt
import datetime
import sys
import torch

In [0]:
torch.__version__

'1.1.0'

In [0]:
!nvidia-smi

Tue May 14 00:02:47 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0    30W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

Thats a good gpu, with no pesky graphics taking up storage either

In [0]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [0]:
%cd /gdrive/My\ Drive/cs445/

/gdrive/My Drive/cs445


In [0]:
import neuralnetworks_pytorch as nn

In [0]:
ls

final_training_decade.ipynb  make_nnet                  pickles/
final_training_make.ipynb    neuralnetworks_pytorch.py  __pycache__/


In [0]:
#Xbatch1 = pkl.load(open("./pickles/training_batches/batch0.pkl", "rb"))
#Tbatch1 = pkl.load(open("./pickles/training_batches/batch0_targets.pkl", "rb"))
#Xbatch2 = pkl.load(open("./pickles/training_batches/batch1.pkl", "rb"))
#Tbatch2 = pkl.load(open("./pickles/training_batches/batch1_targets.pkl", "rb"))
#Xbatch3 = pkl.load(open("./pickles/training_batches/batch2.pkl", "rb"))
#Tbatch3 = pkl.load(open("./pickles/training_batches/batch2_targets.pkl", "rb"))
#Xbatch1_test = pkl.load(open("./pickles/test_batches/batch0.pkl", "rb"))
#Tbatch1_test = pkl.load(open("./pickles/test_batches/batch0_targets.pkl", "rb"))

In [0]:
Xbatch = pkl.load(open("./pickles/train_batches3-5/batch0.pkl", "rb"))
Tbatch = pkl.load(open("./pickles/train_batches3-5/batch0_targets.pkl", "rb"))

In [0]:
Xbatch_test = pkl.load(open("./pickles/test_batches3-5/batch0.pkl", "rb"))
Tbatch_test = pkl.load(open("./pickles/test_batches3-5/batch0_targets.pkl", "rb")) 

In [0]:
Xbatch.shape, Xbatch_test.shape

((1000, 165, 330, 3), (1000, 165, 330, 3))

In [0]:
Xbatch.nbytes #118 MB, used to be 450 MB a batch before downsampling

163350000

In [0]:
makes = np.unique(Tbatch[:, 0])
styles = np.unique(Tbatch[:,1])
years = np.unique(Tbatch[:,2])
years, makes, styles

(array(['1991', '1993', '1994', '1997', '1998', '1999', '2000', '2001',
        '2002', '2006', '2007', '2008', '2009', '2010', '2011', '2012'],
       dtype='<U54'),
 array(['AM', 'Acura', 'Aston', 'Audi', 'BMW', 'Bentley', 'Bugatti',
        'Buick', 'Cadillac', 'Chevrolet', 'Chrysler', 'Daewoo', 'Dodge',
        'Eagle', 'FIAT', 'Ferrari', 'Fisker', 'Ford', 'GMC', 'Geo',
        'HUMMER', 'Honda', 'Hyundai', 'Infiniti', 'Isuzu', 'Jaguar',
        'Jeep', 'Lamborghini', 'Land', 'Lincoln', 'MINI', 'Maybach',
        'Mazda', 'McLaren', 'Mercedes-Benz', 'Mitsubishi', 'Nissan',
        'Plymouth', 'Porsche', 'Ram', 'Rolls-Royce', 'Scion', 'Spyker',
        'Suzuki', 'Tesla', 'Toyota', 'Volkswagen', 'Volvo', 'smart'],
       dtype='<U54'),
 array(['Convertible', 'Coupe', 'Hatchback', 'Minivan', 'SUV', 'Sedan',
        'Truck', 'Van', 'Wagon'], dtype='<U54'))

In [0]:
def makeXandT(Xbatch, Xbatch_test, Tbatch, Tbatch_test, makes):
  for target in Tbatch:
    target[0] = np.where(makes == target[0])[0][0]
    
  for target in Tbatch_test:
    target[0] = np.where(makes == target[0])[0][0]

  # Change to the channels first format, which is what is expected by pytorch  
  Xbatch = np.rollaxis(Xbatch, 3, 1)
  Xbatch_test = np.rollaxis(Xbatch_test, 3, 1)
  # Make target classes for vehicle make
  Ttrain = Tbatch[:, 0]
  Ttest = Tbatch_test[:, 0]
  
  return Xbatch, Xbatch_test, Ttrain, Ttest

In [0]:
def percent_correct(actual, predicted):
    return 100 * np.mean(actual == predicted)

In [0]:
n_outputs = len(makes)

In [0]:
input_size_in_pixels = 3

Image is 55 by 110. For convolution size, we can start with 7x7, then 5x5 down to 3x3

In [0]:
n_hiddens_by_layer_1 = [25, 50, 75, 50, 25]    
n_hiddens_by_layer_2 = [30, 60, 75, 60, 30]    
n_hiddens_by_layer_3 = [10, 10, 10, 10, 100]   
n_hiddens_by_layer_4 = [10, 20, 25, 25, 100]   
n_hiddens_by_layer_5 = [10, 20, 25, 30, 100]   
n_hiddens_by_layer_6 = [20, 35, 50, 60, 200] 
n_hiddens_by_layer_7 = [50, 65, 80, 100, 100]  

batch_size = 25
learning_rate = 0.0001
n_iterations = 5
n_conv_layers = 4

window_1 = [27, 19, 9, 9]
window_2 = [21, 11, 5, 5]
window_3 = [27, 19, 13, 7]
window_4 = [41, 29, 15, 15]

stride = [2, 2, 2, 2]

relu = True
gpu = True

In [0]:
Xtrain, Xtest, Ttrain, Ttest = makeXandT(Xbatch, Xbatch_test, Tbatch, Tbatch_test, makes)

In [0]:
input_height = Xtrain.shape[2]
input_width = Xtrain.shape[3]

In [0]:
nnet = nn.NeuralNetworkClassifier_Pytorch(input_size_in_pixels, 
                                          n_hiddens_by_layer_7, 
                                          n_outputs, 
                                          relu,gpu, 
                                          n_conv_layers, window_1, stride, input_height, input_width)

NeuralNetworkClassifier_Pytorch created on cuda


In [0]:
nnet.train(Xtrain, Ttrain.astype(np.uint8), Xtest, Ttest.astype(np.uint8), n_iterations, batch_size, learning_rate)

Iteration 1, cost = 3.6748, acc = 11.50
Iteration 2, cost = 3.5174, acc = 11.80
Iteration 3, cost = 3.4863, acc = 11.40
Iteration 4, cost = 3.3923, acc = 10.70
Iteration 5, cost = 3.3107, acc = 10.40


In [0]:
Xbatch = pkl.load(open("./pickles/train_batches3-5/batch1.pkl", "rb"))
Tbatch = pkl.load(open("./pickles/train_batches3-5/batch1_targets.pkl", "rb"))
Xbatch_test = pkl.load(open("./pickles/test_batches3-5/batch1.pkl", "rb"))
Tbatch_test = pkl.load(open("./pickles/test_batches3-5/batch1_targets.pkl", "rb")) 

In [0]:
Xtrain, Xtest, Ttrain, Ttest = makeXandT(Xbatch, Xbatch_test, Tbatch, Tbatch_test, makes)
input_height = Xtrain.shape[2]
input_width = Xtrain.shape[3]
nnet.train(Xtrain, Ttrain.astype(np.uint8), Xtest, Ttest.astype(np.uint8), n_iterations, batch_size, learning_rate)

Iteration 1, cost = 3.5329, acc = 10.17
Iteration 2, cost = 3.3383, acc = 9.22
Iteration 3, cost = 3.0742, acc = 11.76
Iteration 4, cost = 2.6506, acc = 8.90
Iteration 5, cost = 2.2449, acc = 10.02


In [0]:
Xbatch = pkl.load(open("./pickles/train_batches3-5/batch2.pkl", "rb"))
Tbatch = pkl.load(open("./pickles/train_batches3-5/batch2_targets.pkl", "rb"))
Xbatch_test = pkl.load(open("./pickles/test_batches3-5/batch0.pkl", "rb"))
Tbatch_test = pkl.load(open("./pickles/test_batches3-5/batch0_targets.pkl", "rb")) 

In [0]:
Xtrain, Xtest, Ttrain, Ttest = makeXandT(Xbatch, Xbatch_test, Tbatch, Tbatch_test, makes)
input_height = Xtrain.shape[2]
input_width = Xtrain.shape[3]
nnet.train(Xtrain, Ttrain.astype(np.uint8), Xtest, Ttest.astype(np.uint8), n_iterations, batch_size, learning_rate)

Iteration 1, cost = 3.6499, acc = 11.70
Iteration 2, cost = 3.3578, acc = 10.60
Iteration 3, cost = 3.0086, acc = 9.50
Iteration 4, cost = 2.5699, acc = 7.70
Iteration 5, cost = 2.2263, acc = 7.20


In [0]:
Xbatch = pkl.load(open("./pickles/train_batches3-5/batch3.pkl", "rb"))
Tbatch = pkl.load(open("./pickles/train_batches3-5/batch3_targets.pkl", "rb"))
Xbatch_test = pkl.load(open("./pickles/test_batches3-5/batch1.pkl", "rb"))
Tbatch_test = pkl.load(open("./pickles/test_batches3-5/batch1_targets.pkl", "rb")) 

In [0]:
Xtrain, Xtest, Ttrain, Ttest = makeXandT(Xbatch, Xbatch_test, Tbatch, Tbatch_test, makes)
input_height = Xtrain.shape[2]
input_width = Xtrain.shape[3]
nnet.train(Xtrain, Ttrain.astype(np.uint8), Xtest, Ttest.astype(np.uint8), n_iterations, batch_size, learning_rate)

Iteration 1, cost = 3.6247, acc = 10.49
Iteration 2, cost = 3.3410, acc = 9.86
Iteration 3, cost = 2.9952, acc = 10.02
Iteration 4, cost = 2.5489, acc = 7.31
Iteration 5, cost = 1.9675, acc = 8.27


In [0]:
Xbatch = pkl.load(open("./pickles/train_batches3-5/batch4.pkl", "rb"))
Tbatch = pkl.load(open("./pickles/train_batches3-5/batch4_targets.pkl", "rb"))
Xbatch_test = pkl.load(open("./pickles/test_batches3-5/batch0.pkl", "rb"))
Tbatch_test = pkl.load(open("./pickles/test_batches3-5/batch0_targets.pkl", "rb")) 

In [0]:
Xtrain, Xtest, Ttrain, Ttest = makeXandT(Xbatch, Xbatch_test, Tbatch, Tbatch_test, makes)
input_height = Xtrain.shape[2]
input_width = Xtrain.shape[3]
nnet.train(Xtrain, Ttrain.astype(np.uint8), Xtest, Ttest.astype(np.uint8), n_iterations, batch_size, learning_rate)

Iteration 1, cost = 3.6586, acc = 11.50
Iteration 2, cost = 3.2326, acc = 8.40
Iteration 3, cost = 2.8573, acc = 7.30
Iteration 4, cost = 2.3284, acc = 8.70
Iteration 5, cost = 1.9550, acc = 8.90


In [0]:
Xbatch = pkl.load(open("./pickles/train_batches3-5/batch5.pkl", "rb"))
Tbatch = pkl.load(open("./pickles/train_batches3-5/batch5_targets.pkl", "rb"))
Xbatch_test = pkl.load(open("./pickles/test_batches3-5/batch1.pkl", "rb"))
Tbatch_test = pkl.load(open("./pickles/test_batches3-5/batch1_targets.pkl", "rb")) 

In [0]:
Xtrain, Xtest, Ttrain, Ttest = makeXandT(Xbatch, Xbatch_test, Tbatch, Tbatch_test, makes)
input_height = Xtrain.shape[2]
input_width = Xtrain.shape[3]
nnet.train(Xtrain, Ttrain.astype(np.uint8), Xtest, Ttest.astype(np.uint8), n_iterations, batch_size, learning_rate)

Iteration 1, cost = 3.7530, acc = 9.54
Iteration 2, cost = 3.2194, acc = 9.54
Iteration 3, cost = 2.6927, acc = 7.47
Iteration 4, cost = 2.4545, acc = 7.31
Iteration 5, cost = 1.7842, acc = 7.31


In [0]:
Xbatch = pkl.load(open("./pickles/train_batches3-5/batch6.pkl", "rb"))
Tbatch = pkl.load(open("./pickles/train_batches3-5/batch6_targets.pkl", "rb"))
Xbatch_test = pkl.load(open("./pickles/test_batches3-5/batch0.pkl", "rb"))
Tbatch_test = pkl.load(open("./pickles/test_batches3-5/batch0_targets.pkl", "rb")) 

In [0]:
Xtrain, Xtest, Ttrain, Ttest = makeXandT(Xbatch, Xbatch_test, Tbatch, Tbatch_test, makes)
input_height = Xtrain.shape[2]
input_width = Xtrain.shape[3]
nnet.train(Xtrain, Ttrain.astype(np.uint8), Xtest, Ttest.astype(np.uint8), n_iterations, batch_size, learning_rate)

Iteration 1, cost = 3.8862, acc = 7.60
Iteration 2, cost = 3.2835, acc = 9.20
Iteration 3, cost = 2.8579, acc = 8.70
Iteration 4, cost = 2.2563, acc = 8.90
Iteration 5, cost = 1.5949, acc = 9.70


In [0]:
classes, _, _ = nnet.use(Xtest)
percent = percent_correct(Ttest.astype(np.uint8), classes)
percent


9.700000000000001

In [0]:
cpu_nnet = nnet.to(torch.device('cpu'))
cpu_nnet.device = 'cpu'

In [0]:
pkl.dump(cpu_nnet, open("make_nnet", "wb"))